In [21]:


# completion = client.chat.completions.create(
#   model="gpt-3.5-turbo",
#   messages=[
#     {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
#     {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
#   ],
#   max_tokens=50,
# )



In [1]:
import ast

def remove_docstring(source_code):
    # Parse the source code into an AST
    parsed_code = ast.parse(source_code)

    # Function to remove the docstring
    class DocstringRemover(ast.NodeTransformer):
        def visit_FunctionDef(self, node):
            # If there's a docstring, it's the first node in the body
            # that is an ast.Expr with a value of type ast.Str
            if (node.body and isinstance(node.body[0], ast.Expr) and 
                isinstance(node.body[0].value, ast.Str)):
                # Remove the docstring
                node.body.pop(0)
            return node

    # Use the transformer to remove the docstring
    transformer = DocstringRemover()
    modified_tree = transformer.visit(parsed_code)

    # Convert the modified AST back to source code
    return ast.unparse(modified_tree)


In [2]:
import json
import sys
from concurrent.futures import ThreadPoolExecutor

sys.path.append('/media/data/flowers/OpenELM')
from key import key
from openai import OpenAI
client = OpenAI(api_key=key,max_retries=5,timeout=50)

cfg_generation={"temperature": 0.,
            "top_p": 1,
            # TODO: rename config option?
            "model": "gpt-3.5-turbo",
            "max_tokens":200,
}

def get_completion(client, prompt : str, cfg_generation, tools=None)->str:
    """Get completion from OpenAI API"""
    
    flag_tool=tools is not None
    if flag_tool:
        cfg_generation.update({"tools": tools})
        tool_name=tools[0]["function"]["name"]
        cfg_generation.update({"tool_choice": {"type": "function", "function": {"name": tool_name}}})
    try :
        completion = client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You are an AI programming assistant"},#You are a coding assistant, skilled in writting code with creative flair."},
            {"role": "user", "content": prompt}
        ],**cfg_generation
        )
    except Exception as e:
        print("completion problem: ",e)
        return None 
    # completion_token = completion.usage.completion_tokens
    # prompt_token = completion.usage.prompt_tokens
    
    if flag_tool:
        try:
            tool_out=out.choices[0].message.tool_calls[0].function.arguments
            return eval(tool_out)
        except Exception as e:  
            print("tool parsing problem: ",e)
            return None
        
    out = completion.choices[0].message.content
    return out

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

def get_multiple_completions(client, batch_prompt: list[str], cfg_generation: dict, batch_tools: list[list[dict]]=None,max_workers=20)->list[str]:
    """Get multiple completions from OpenAI API
    batch_tools =[[tools]] tools is the function, toll_name is the name of the tool
    """
    
    completions = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for sub_batch in chunks(batch_prompt, max_workers):
            for idx,message_list in enumerate(sub_batch):
                # kwargs_modified = args.copy()
                # kwargs_modified["messages"] = message_list
                kwargs = {"client":client, "prompt":message_list}
                kwargs["cfg_generation"]=cfg_generation
                # if "kwargs" in kwargs_modified:
                #     original_kwargs = kwargs_modified.pop("kwargs")
                future = executor.submit(
                    get_completion,**kwargs
                )
                completions.append(future)

    # Retrieve the results from the futures
    results = [future.result() for future in completions]
    return results

In [40]:
# batch_prompt=["Compose a poem that explains the concept of recursion in programming."]*3

# out=get_multiple_completions(client, batch_prompt, cfg_generation)

In [73]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "poem",
            "description": "Use this function to answer user questions about poem.",
            "parameters": {
                "type": "object",
                "properties": {
                    "write_poem": {
                        "type": "string",
                        "description": f"""
                                Write a short poem.
                                """,
                    }
                },
                "required": ["write_poem"],
            },
        }
    }
]

tool_choice={"type": "function", "function": {"name": "poem"}}

In [74]:

out=get_completion(client, batch_prompt[0], cfg_generation,tools=tools,tool_choice=tool_choice)

In [75]:
tool_out=out.choices[0].message.tool_calls[0].function.arguments
print(eval(tool_out))

SyntaxError: unterminated string literal (detected at line 2) (<string>, line 2)

In [60]:
print(tool_out)

{
  "write_poem": "Recursion, a concept profound,\nIn programming, it can be found.\nA function that calls itself, you see,\nA loop within, a mystery.\n\n


In [17]:
def get_completion(client, prompt,max_tokens=50,tools=None):
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a coding assistant, skilled in writting code"},# with creative flair."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=max_tokens,
    )
    completion_token = completion.usage.completion_tokens
    prompt_token = completion.usage.prompt_tokens
    out = completion.choices[0].message.content
    return out

In [20]:
batch_prompt=["Compose a poem that explains the concept of recursion in programming."]*10

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]
        
max_workers=100
completions = []

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    for sub_batch in chunks(batch_prompt, max_workers):
        for idx,message_list in enumerate(sub_batch):
            # kwargs_modified = args.copy()
            # kwargs_modified["messages"] = message_list
            kwargs = {"max_tokens":100, "client":client, "prompt":message_list}
            # if "kwargs" in kwargs_modified:
            #     original_kwargs = kwargs_modified.pop("kwargs")
            future = executor.submit(
                get_completion,**kwargs
            )
            completions.append(future)

# Retrieve the results from the futures
results = [future.result() for future in completions]
for i in results:
    print(i)
    print("\n === \n")

In a realm of code and algorithms deep,
Where patterns reside and wonders creep,
There lies a concept, a methodical delight,
A mesmerizing dance known as recursion's flight.

Just like a mirror reflecting its own reflection,
Recursion unveils a coder's introspection,
A task of utmost elegance, looping within,
Unraveling the mysteries, where the magic begins.

With a function's call, it starts with grace,
Delving into a world both familiar and base,
Dividing the

 === 

In the realm of lines and loops, we tread,
Where code we write, where algorithms spread,
A concept so profound, with elegance,
It's time to explore the depths of recursion.

Imagine a function, oh so enchanting,
That calls itself, like echoes never-ending,
A stunning dance with its own reflection,
Recursion, a powerful manifestation.

Just like a mirror within a mirror,
Layers upon layers, clearer and clearer,
A task divided, into fragments so small,
Until the base case

 === 

In the world of code, where logic resides,


In [18]:
rng = np.random.default_rng()
shuffled_indices = np.arange(10)
rng.random.shuffle(shuffled_indices)

AttributeError: 'builtin_function_or_method' object has no attribute 'shuffle'

In [3]:
from itertools import combinations

# List of skills (assuming they are just numbered for simplicity)
skills = list(range(1, 21))

# Generate all combinations of up to 5 skills
skill_combinations = set()
for r in range(1, 6):  # From 1 skill to 5 skills
    skill_combinations.update(combinations(skills, r))

# Checking the first few combinations and the total number of combinations
sample_combinations = list(skill_combinations)[:5]
total_combinations = len(skill_combinations)
sample_combinations, total_combinations


([(1, 5, 9, 17),
  (5, 8, 16, 17, 19),
  (6, 8, 12, 13),
  (1, 9, 10, 18, 19),
  (2, 4, 5)],
 21699)

In [4]:
import numpy as np
dists=np.array([0,0,2,0,5,2])
shuffled_indices = np.arange(len(dists))
np.random.shuffle(shuffled_indices)
nearest_niches = shuffled_indices[np.argsort(dists[shuffled_indices])]

In [5]:
nearest_niches

array([1, 0, 3, 5, 2, 4])

In [6]:
a=[1,2,3]
b=[4,5,6]
a.extend(b)
a

[1, 2, 3, 4, 5, 6]

In [8]:
a=[1,2,3,4,5,6,7,8,9,10]
np.random.choice(a)

8

In [1]:
import json
import sys
sys.path.append('/media/data/flowers/OpenELM')
from key import key
from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage
import numpy as np  
import tiktoken
import matplotlib.pyplot as plt
enc = tiktoken.encoding_for_model("gpt-4")

base_path="/home/flowers/work/OpenELM/run_saved/all_json/full_trainset/"
all_path_baseline_template=["maps_{i}_rd_gen.json","maps_{i}_elm.json","maps_{i}_elm_NLP.json","maps_{i}_imgep_random.json"]#,"maps_{i}_imgep_smart.json"]
all_path_baseline_template= [base_path+i for i in all_path_baseline_template]
all_path_all_emb_seed=[[i_path.format(i=idx+1) for i_path in all_path_baseline_template] for idx in range(len(all_path_baseline_template))]
path_rd_gen = all_path_all_emb_seed[0][0]
path_imgep = all_path_all_emb_seed[0][3]

cfg: dict = {
    "openai_api_key": key,
    "max_tokens": 512,
    "temperature": 0.0,
    "top_p": 1.,
    # TODO: rename config option?
    "model_name": "gpt-3.5-turbo-1106",#"gpt-3.5-turbo-0613",#gpt-3.5-turbo-1106
    "request_timeout": 40,
}
chatGPT = ChatOpenAI(**cfg)    
def gen_response(prompts):
    #check prompts is list of strings
    if isinstance(prompts, str):
        prompts=[prompts]
    if len(prompts)>100:
        raise ValueError("Too many prompts")

    len_prompts = len(prompts)
    prompts = [[HumanMessage(content=message)] for message in prompts]
    out=chatGPT.generate(prompts)#.generations#[0][0].text
    out_gen=out.generations
    len_out=len(out_gen)
    assert len_out==len_prompts, "len_out != len_prompts"
    responses=[out_gen[i][0].text for i in range(len_out)]
    return responses

import ast
import os
def exctract_docstring(source_code: str, remove_docstring : bool = True) -> str:
    """ 
    remove docstring of function f in source_code
    if remove_docstring == False, just copy docstring and return the unmodified source_code 
    """
    puzzle_formated= source_code

    # Parse the source code into an AST
    tree = ast.parse(source_code)

    # Extract the docstring from function f and remove it
    f_docstring = None
    for item in tree.body:
        if isinstance(item, ast.FunctionDef) and item.name == 'f':
            if ast.get_docstring(item):
                f_docstring = ast.get_docstring(item)
                if (f_docstring != None):
                    # delimiters ="example","Example","For example","Example:"
                    # regex_pattern = '|'.join(map(re.escape, delimiters))
                    # f_docstring_split = re.split(regex_pattern, f_docstring)[0]
                    # if replace docstring with f_docstring_split
                    # item.body[0].value.s = f_docstring_split
                    # just remove docstring
                    if remove_docstring:
                        # if (item.body and isinstance(item.body[0], ast.Expr) and 
                        #     isinstance(item.body[0].value, ast.Str)):
                        item.body.pop(0) # remove docstring
                            # item.body[0].value.s = ""
                    
    if (f_docstring != None and remove_docstring):
        # Convert the modified AST back to source code
        puzzle_formated=ast.unparse(tree)
    # puzzle_formated=puzzle_formated.replace('""""""',"")
    puzzle_formated = os.linesep.join([s for s in puzzle_formated.splitlines() if s.strip()]) # remove empty line

    return puzzle_formated,f_docstring

/home/flowers/miniconda3/envs/codegpt/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


In [ ]:

from openai import OpenAI
client = OpenAI(api_key= key)
# prompts = ["Once upon a time, in America","Once upon a time, in France","Once upon a time, in Japan"]
num_stories = 10
prompts = ["Once upon a time,"] * num_stories

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=prompts,
#  logprobs=True,
 max_tokens=20,
)


In [23]:
completion.__dict__

{'id': 'chatcmpl-8fsHFnd0mFZ1eC3miWDhLMxyLr6pK',
 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))],
 'created': 1704990781,
 'model': 'gpt-3.5-turbo-0613',
 'object': 'chat.completion',
 'system_fingerprint': None,
 'usage': CompletionUsage(completion_tokens=9, prompt_tokens=19, total_tokens=28)}

In [3]:
import instructor
from openai import OpenAI
from pydantic import BaseModel
from key import key
# This enables response_model keyword
# from client.chat.completions.create
client = instructor.patch(OpenAI(api_key= key))

class UserDetail(BaseModel):
    name: str
    age: int

user = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=UserDetail,
    messages=[
        {"role": "user", "content": "Extract Jason is 25 years old"},
    ]
)

assert isinstance(user, UserDetail)
assert user.name == "Jason"
assert user.age == 25


NameError: name 'batch_prompt' is not defined

In [9]:
batch_prompt=["Jason is 25"]*10
max_workers=10
temperature=0.0
completions=[]
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    for sub_batch in chunks(batch_prompt, max_workers):
        for idx,message_list in enumerate(sub_batch):
            # kwargs_modified = args.copy()
            # kwargs_modified["messages"] = message_list
            kwargs = {"model":"gpt-3.5-turbo", "response_model":UserDetail,
                      "messages":[
        {"role": "user", "content": f"Extract {message_list} years old"},
    ]}
            # kwargs["cfg_generation"]=cfg_generation
            if temperature is not None:
                kwargs["temperature"]= temperature
            # if "kwargs" in kwargs_modified:
            #     original_kwargs = kwargs_modified.pop("kwargs")
            
            future = executor.submit(
                client.chat.completions.create,**kwargs
            )
            completions.append(future)
# Retrieve the results from the futures
results = [future.result() for future in completions]

In [ ]:
list1=[1,2,3,4,5,6]
list2=[4,5,6,7,8,9]

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

In [10]:
results

[UserDetail(name='Jason', age=25),
 UserDetail(name='Jason', age=25),
 UserDetail(name='Jason', age=25),
 UserDetail(name='Jason', age=25),
 UserDetail(name='Jason', age=25),
 UserDetail(name='Jason', age=25),
 UserDetail(name='Jason', age=25),
 UserDetail(name='Jason', age=25),
 UserDetail(name='Jason', age=25),
 UserDetail(name='Jason', age=25)]

In [5]:
user.__dict__

{'name': 'Jason',
 'age': 25,
 '_raw_response': ChatCompletion(id='chatcmpl-8jmbhP7UGgpim6Ku5HSJNOr1qVlcj', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "name": "Jason",\n  "age": 25\n}', name='UserDetail'), tool_calls=None))], created=1705922297, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=72, total_tokens=88))}

# exctract data to compute quality

In [2]:
def exctract_subset_order_by_gen(data,sample_max=10,gen2sample=None):
    list_idx=[i["idx_generation"] for i in data]
    gen_max=np.max(list_idx)
    list_data_filtered=[]
    if gen2sample is None:
        gen2sample = list(np.arange(1,gen_max+1,50))
        if not(gen_max in gen2sample):
            gen2sample.append(gen_max)
# Filtering and sampling
    list_data_filtered = []
    for gen in gen2sample:
        # Filter data for each generation
        filtered_data = [i for i in data if i["idx_generation"] == gen]
        print(len(filtered_data))
        # Sample data if more than 'sample_max' items are available
        if len(filtered_data) > sample_max:
            sampled_data = np.random.choice(filtered_data, sample_max, replace=False)
        else:
            sampled_data = filtered_data
        
        list_data_filtered.extend(sampled_data)
    return list_data_filtered

path_train="/home/flowers/work/OpenELM/run_saved/all_json/P3_train.json"
with open(path_train) as f:
    data_train = json.load(f)
data_train_subset =  np.random.choice(data_train, 50, replace=False)

with open(path_rd_gen) as f:
    data_rdgen = json.load(f)

with open(path_imgep) as f:
    data_imgep = json.load(f)

sample_max=6
data_rdgen_subsample = exctract_subset_order_by_gen(data_rdgen,sample_max)
data_imgep_subsample = exctract_subset_order_by_gen(data_imgep,sample_max)

len(data_rdgen_subsample),len(data_imgep_subsample)

14
19
14
19
19
16
17
20
11
15
16
6
13
9
5
5
4
9
10
11
14
10


(66, 62)

# analyse quality test

In [3]:
def analyse_quality(puzzle,gen_response=False): # format => style?
    puzzle_formated, f_docstring =exctract_docstring(puzzle)

#     promptbase= '''Rate the quality of the following Python Programming Puzzle on a scale from 1 to 10.
# You should pay a particular attention to the function g as it should be the solution to the problem.
# The puzzle must be in the following format:
 
# Problem: 
# A few sentences to explain the problem ( without the solution )

# ```python
# def f(solution,args) -> bool:
#     # Python code to test the solution return by g 
#     # This function is a test units for the solution return by g, it must return True if the solution is correct and False otherwise
    
# def g(args) -> solution:
#     # Python code to solve the problem
#     # It should generalise to all possible args 
#     return solution 

# assert f(g()) == True
# ```
# '''
#     promptbase+=f'''Answer in JSON with the key 'grade'. Here is the puzzle to evaluate:
# Problem:
# {f_docstring}

# ```
# {puzzle_formated}
# ```
# Take a deep breath and work on this problem step-by-step.'''

    prompt='''You need to rate the quality of the following Python Programming Puzzle on a scale from 0 to 10.
You should pay a particular attention to the function g as it should be the solution to the problem. And not just a function that gives arguments to f.
Puzzles must follow this format:
Problem: 
A few sentences to explain the problem ( without the solution )

```python
def f(solution,args) -> bool:
    # Python code to test the solution return by g 
    # This function is a test unit for the solution returned by g, it must return True if the solution is correct and False otherwise
    
def g(args) -> solution:
    # Python code to solve the problem
    # It should generalize to all possible args 
    return solution 

assert f(g()) == True
```

'''
    prompt+=f'''Here is the puzzle to grade:
Problem:
{f_docstring}

```
{puzzle_formated}
```
First, write a few sentences about the quality of the functions f and g in a few sentences.
Then assign a grade on the 'quality' of the function f and the function g (0 to 10).'''
    prompt+=" Then summarize your answer in JSON format (example: {'quality_f': integer (0 to 10),'quality_g': integer (0 to 10)})"

    #Take a deep breath and work on this problem step-by-step. Then provide in JSON format with the key 'grade' the quality and with the key 'format': True if the function g really solve the problem or False if it just use g to generate arguments. 
    if gen_response:
        return prompt,gen_response([prompt])
    else:
        return prompt

def extract_grade(response : str):
    """
    extract grade from response in JSON format
    """ 
    quality_f,quality_g=-1,-1
    if response.count("{")==1 and response.count("}")==1:    
        dic_extracted="{"+response.split("{")[-1].split("}")[0]+"}"
        try:
            dic_good_format = eval(dic_extracted)
            if "quality_f" in dic_good_format:
                quality_f=dic_good_format["quality_f"]
            if "quality_g" in dic_good_format:
                quality_g=dic_good_format["quality_g"]
        except:
            print("bad format")
    return (quality_f,quality_g)

In [4]:
def compute_quality(data):
    # data is a list of dict
    list_prompt=[]
    for puzzle in data:
        prompt=analyse_quality(puzzle["program_str"],gen_response=False)
        list_prompt.append(prompt)
    list_response = gen_response(list_prompt)
    for idx,puzzle in enumerate(data):
        (puzzle["quality_f"],puzzle["quality_g"])=extract_grade(list_response[idx])

In [5]:
# len(data_imgep_subsample)
compute_quality(data_imgep_subsample[:1])

In [10]:
compute_quality(data_imgep_subsample[:8])

In [8]:
data_imgep_subsample[:8]

[{'fitness': 1.0,
  'program_str': 'from typing import *\ndef f(points: List[Tuple[int, int]], threshold=3) -> bool:\n    """Check if there exists a straight line passing through at least threshold number of points"""\n    n = len(points)\n    if n < threshold:\n        return False\n    for i in range(n - 2):\n        (x1, y1) = points[i]\n        (x2, y2) = points[i + 1]\n        (x3, y3) = points[i + 2]\n        if (y2 - y1) * (x3 - x2) != (y3 - y2) * (x2 - x1):\n            return False\n    return True\ndef g(points=[(1, 2), (3, 4), (5, 6), (7, 8)], threshold=3):\n    return points\nassert f(g())',
  'emb': [1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0],
  'idx_generation': 1,
  'target_skills': [1, 1, 0, 1, 0, 0, 1, 0, 0, 1],
  'emb_features': [0.0279541015625,
   -0.00872802734375,
   -0.12286376953125,
   0.0072174072265625,
   -0.0888671875,
   -0.01263427734375,
   -0.088623046875,
   -0.04205322265625,
   -9.34600830078125e-05,
   -0.004596710205078125,
   -0.05783081054

In [44]:
compute_quality(data_imgep_subsample)
compute_quality(data_rdgen_subsample)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised APIError: The server had an error while processing your request. Sorry about that! {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that!",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error while processing your request. Sorry about that!', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Tue, 09 Jan 2024 16:52:52 GMT', 'Content-Type': 'application/json', 'Content-Length': '176', 'Connection': 'keep-alive', 'access-control-allow-origin': '*', 'openai-model': 'gpt-3.5-turbo-1106', 'openai-organization': 'user-95sh09uepewba9olgw4gf5fl', 'openai-processing-ms': '7818', 'openai-version': '2020-10-01', 'strict-transport-security': 'max-age=15724800; includeSubDomains', 'x-ratelimit-limit-requests': '10000', 'x-ratelimit-limit-to

In [61]:
path_train = "/home/flowers/work/OpenELM/notebook/P3_train.json"
with open(path_train) as f:
    data_train = json.load(f)

len(data_train)
#sample 100 puzzles
np.random.seed(42)
data_train_subsample =  np.random.choice(data_train, 100, replace=False)
compute_quality(data_train_subsample)

In [19]:
te="""The function f is well-written and effectively checks if the given matrix is a Toeplitz matrix. It correctly iterates through the matrix and compares each element with the one below and to the right of it. The function g simply returns the input matrix, which is not a solution to the problem but rather just a way to pass arguments to f.

quality_f: 8
quality_g: 3

{'quality_f': 8, 'quality_g': 3}
"""
quality_f,quality_g=-1,-1
if te.count("{")==1 and te.count("}")==1:    
    dic_extracted="{"+te.split("{")[-1].split("}")[0]+"}"
    try:
        dic_good_format = eval(dic_extracted)
        if "quality_f" in dic_extracted:
            quality_f=dic_extracted["quality_f"]
        if "quality_g" in dic_extracted:
            quality_g=dic_extracted["quality_g"]
    except:
        print("bad format")
# return quality_f,quality_g

9.5

In [ ]:
prompt,reponse=analyse_quality(data_imgep_subsample[-1]["program_str"])
out_0 = prompt+"\n"+reponse[0]
print(out_0)
print("n token used = ",len(enc.encode(out_0)))
print("\n=====\n\n")
prompt,reponse=analyse_quality_g(data_imgep_subsample[-1]["program_str"])
out_1 = prompt+"\n"+reponse[0]
print(out_1)
print("n token used = ",len(enc.encode(out_1)))


# plot results

In [ ]:
def extract_qualities(data, key=['quality_g']):
    """Extract and filter qualities from the data."""
    qualities = [np.mean([d[k] for k in key]) for d in data ]# if d[key] != -1]
    return [q for q in qualities if not np.isnan(q)]
path_save_fig="/home/flowers/work/OpenELM/notebook/fig/"
possible_key= [['quality_f'],['quality_g'], ['quality_f', 'quality_g']]
for key in possible_key: #key=['quality_g']
    y_imgep = extract_qualities(data_imgep_subsample,key=key)
    y_rdgen = extract_qualities(data_rdgen_subsample,key=key)
    y_train= extract_qualities(data_train_subsample,key=key)


    # Determine common bin settings for both histograms
    # bins = np.linspace(min(y_imgep + y_rdgen+y_train), max(y_imgep + y_rdgen+y_train), 20)

    # Plotting
    plt.hist([y_imgep, y_rdgen,y_train],  label=["imgep", "rdgen","P3 train"], alpha=0.7, density=True,align='left',)
    plt.legend()
    plt.xlabel("Quality Score")
    plt.ylabel("Frequency")
    title_key="("
    for i in key:
        title_key+= i +" + "
    
    title_key=title_key[:-3]+")"
    plt.title("Quality Score Distribution "+title_key)
    plt.grid(True)
    plt.xticks(np.arange(-1, 11, 1))  
    plt.savefig(path_save_fig+"quality_score_distribution_"+title_key+".png")
    plt.show()

# bins = np.linspace(min(y_imgep + y_rdgen), max(y_imgep + y_rdgen), 20)

# Plotting
# plt.hist([y_imgep, y_rdgen], bins=bins, label=["imgep", "rdgen","train"], alpha=0.7)
# plt.legend()
# plt.xlabel("Quality Score")
# plt.ylabel("Frequency")
# plt.title("Quality Score Distribution")
# plt.grid(True)
# plt.show()


In [50]:
def print_data(data, grade=3,key='quality_g'):
    """Extract and filter data given qualitie score."""
    data_filtered = [d["program_str"] for d in data if d[key] == grade]
    for i in data_filtered:
        print(i)
        txt=key[0]+" = "+str(grade)
        print(f"\n ===== {txt} ===== \n")


In [ ]:
list_data=[data_imgep_subsample,data_rdgen_subsample,data_train_subsample]
idx=2
print_data(list_data[idx],grade=8,key='quality_g')

In [17]:
base_path="/home/flowers/work/OpenELM/notebook/"
#save json
# with open(base_path+"data_imgep_subsample.json", 'w') as f:
#     json.dump(data_imgep_subsample, f)

# with open(base_path+"data_rdgen_subsample.json", 'w') as f:
#     json.dump(data_rdgen_subsample, f)

# with open(base_path+"data_train_subsample.json", 'w') as f:
#     json.dump(list(data_train_subsample), f)

#load json
with open(base_path+"data_imgep_subsample.json") as f:
    data_imgep_subsample = json.load(f)

with open(base_path+"data_rdgen_subsample.json") as f:
    data_rdgen_subsample = json.load(f)

with open(base_path+"data_train_subsample.json") as f:
    data_train_subsample = json.load(f)

In [54]:
import re
text="""In this specific example, the graph has 5 nodes. Node 0 can reach nodes 1 and 2, node 1 can reach node 2, node 2 can reach nodes 3 and 4, node 3 can reach node 4, and node 4 has no outgoing edges.

The solution does not provide any information about the start and end nodes, so it is not clear how the problem of finding a path from start to end can be solved using this solution alone. It seems that the solution is incomplete and requires additional code to determine if a path exists between the start and end nodes.

Based on the given information, I would rate the quality of the solution as 3 out of 10. It is incomplete and lacks clarity.

{"quality": 10}

"""
# Regular expression pattern
def exctract_quality(text):
    pattern = r'\{"quality": \d+\}'

    # Search for the pattern in the text
    match = re.search(pattern, text)

    # Extract and print the match if it exists
    if match:
        extracted_text = match.group()
        return eval(extracted_text)["quality"]
    else:
        return None


10

In [2]:
import json
path_test="/home/flowers/work/OpenELM/old/run/P3_test_emb_wizard3B.json"
with open(path_test) as f:
    data_test = json.load(f)

len(data_test)

981

In [5]:
import pydantic
pydantic.__version__

'2.5.3'

# example of prompt (Generation)


In [ ]:
# from textbook github https://github.com/jina-ai/textbook
'''
Create a code completion exercise on the intersection of “{topic_1.topic}” and “{topic_2.topic}”.  
Write it for a {profession}. 

The exercise must be of the style: 

```
def name(args):

"""Docstring explaining the exercise"""

python code to solve the exercise
```

NO CLASSES

MAKE IT VERY DIFFICULT
'''

# TinyGSM (https://arxiv.org/pdf/2312.09241.pdf)
# We prompt GPT-3.5-turbo to generate problem variants similar to a given question (but not the solution) randomly
'''
Consider the following grade - school math problem : {{ question }}
Generate 10 different math problems similar to this math problem .
- Make sure each question uses diverse NLP and includes multiple logical steps .
- After each generated problem , write down a ** detailed and complete Python program ** to solve the question ** step
by step ** ( do NOT give the result directly , ** DO NOT write calculations in the comments **) .
- The program should contain multiple lines of code and end with ’ result = XXX ’ ( Make sure to replace XXX with the
actual result of the python program ) .
- Make sure your Python program is complete and solves the problem . Do ** NOT ** write things like ’ solution to be
completed ’ , result = ? , insert your code here etc .
- Give the complete solution to solve the problem , written in Python . Do not write things like ’ insert your code
here ’.
- In each new question , ** first end with <| endofquestion | >** , and then start writing the program . Each program
should end with <| endofprogram | >.
- Example format : Question X : New question ( at least 4 sentences long and use diverse NLP ) ( without the solution )
<| endofquestion | > Complete python code with entire solutions and the correct indent ( <| endofprogram | >])
'''

# wizard  https://github.com/nlpxucan/WizardLM/tree/main/Evol_Instruct
base_instruction = "I want you act as a Prompt Rewriter.\r\n \
					Your objective is to rewrite a given prompt into a more complex version to make those famous AI systems (e.g., chatgpt and GPT4) a bit harder to handle.\r\n \
					But the rewritten prompt must be reasonable and must be understood and responded by humans.\r\n \
					Your rewriting cannot omit the non-text parts such as the table and code in #The Given Prompt#:. Also, please do not omit the input in #The Given Prompt#. \r\n \
					You SHOULD complicate the given prompt using the following method: \r\n\
					{} \r\n\
					You should try your best not to make the #Rewritten Prompt# become verbose, #Rewritten Prompt# can only add 10 to 20 words into #The Given Prompt#. \r\n\
					'#The Given Prompt#', '#Rewritten Prompt#', 'given prompt' and 'rewritten prompt' are not allowed to appear in #Rewritten Prompt#\r\n"
def createConstraintsPrompt(instruction):
	prompt = base_instruction.format("Please add one more constraints/requirements into #The Given Prompt#'")
	prompt += "#The Given Prompt#: \r\n {} \r\n".format(instruction)
	prompt += "#Rewritten Prompt#:\r\n"
	return prompt

def createDeepenPrompt(instruction):
	prompt = base_instruction.format("If #The Given Prompt# contains inquiries about certain issues, the depth and breadth of the inquiry can be increased.")
	prompt += "#The Given Prompt#: \r\n {} \r\n".format(instruction)
	prompt += "#Rewritten Prompt#:\r\n"
	return prompt

def createConcretizingPrompt(instruction):
	prompt = base_instruction.format("Please replace general concepts with more specific concepts.")
	prompt += "#The Given Prompt#: \r\n {} \r\n".format(instruction)
	prompt += "#Rewritten Prompt#:\r\n"
	return prompt


def createReasoningPrompt(instruction):
	prompt = base_instruction.format("If #The Given Prompt# can be solved with just a few simple thinking processes, you can rewrite it to explicitly request multiple-step reasoning.")
	prompt += "#The Given Prompt#: \r\n {} \r\n".format(instruction)
	prompt += "#Rewritten Prompt#:\r\n"
	return prompt

# Wizard coder

base_prompt = """Please increase the difficulty of the given programming test question a
bit.

You can increase the difficulty using, but not limited to, the following
methods:
{method}

{question}"""

# Here, {question} represents the current code instruction awaiting evolution, and {method} is the type
# of evolution. The five types we used are listed as follows:

methods =["""Add new constraints and requirements to the original problem, adding
approximately 10 additional words.""",

"""Replace a commonly used requirement in the programming task with a less
common and more specific one.""",

"""If the original problem can be solved with only a few logical steps,
please add more reasoning steps.""",

"""Provide a piece of erroneous code as a reference to increase
misdirection.""",

"""Propose higher time or space complexity requirements, but please refrain
from doing so frequently."""]

In [ ]:
question = """
In preparation for her party , Sarah buys 10 trays of food and 8 cases of beverages. Each tray costs $50 and each case of beverages costs $20.
What is the total cost of the trays and beverages ?
"""
prompt_gsm8k=f'''
Consider the following grade - school math problem : {question}
Generate 10 different math problems similar to this math problem .
- Make sure each question uses diverse NLP and includes multiple logical steps .
- After each generated problem , write down a ** detailed and complete Python program ** to solve the question ** step
by step ** ( do NOT give the result directly , ** DO NOT write calculations in the comments **) .
- The program should contain multiple lines of code and end with ’ result = XXX ’ ( Make sure to replace XXX with the
actual result of the python program ) .
- Make sure your Python program is complete and solves the problem . Do ** NOT ** write things like ’ solution to be
completed ’ , result = ? , insert your code here etc .
- Give the complete solution to solve the problem , written in Python . Do not write things like ’ insert your code
here ’.
- In each new question , ** first end with <| endofquestion | >** , and then start writing the program . Each program
should end with <| endofprogram | >.
- Example format : Question X : New question ( at least 4 sentences long and use diverse NLP ) ( without the solution )
<| endofquestion | > Complete python code with entire solutions and the correct indent ( <| endofprogram | >])
'''
prompt_gsm8k

In [11]:
from itertools import combinations
import numpy as np
# List of skills (assuming they are just numbered for simplicity)
skills = list(range(1, 21))

# Generate all combinations of up to 5 skills
skill_combinations = []
for r in range(1, 6):  # From 1 skill to 5 skills
    skill_combinations.extend(combinations(skills, r))

# Number of combinations
num_combinations = len(skill_combinations)
num_combinations


21699

In [17]:
list(combinations(skills, r))

[(1, 2, 3, 4, 5),
 (1, 2, 3, 4, 6),
 (1, 2, 3, 4, 7),
 (1, 2, 3, 4, 8),
 (1, 2, 3, 4, 9),
 (1, 2, 3, 4, 10),
 (1, 2, 3, 4, 11),
 (1, 2, 3, 4, 12),
 (1, 2, 3, 4, 13),
 (1, 2, 3, 4, 14),
 (1, 2, 3, 4, 15),
 (1, 2, 3, 4, 16),
 (1, 2, 3, 4, 17),
 (1, 2, 3, 4, 18),
 (1, 2, 3, 4, 19),
 (1, 2, 3, 4, 20),
 (1, 2, 3, 5, 6),
 (1, 2, 3, 5, 7),
 (1, 2, 3, 5, 8),
 (1, 2, 3, 5, 9),
 (1, 2, 3, 5, 10),
 (1, 2, 3, 5, 11),
 (1, 2, 3, 5, 12),
 (1, 2, 3, 5, 13),
 (1, 2, 3, 5, 14),
 (1, 2, 3, 5, 15),
 (1, 2, 3, 5, 16),
 (1, 2, 3, 5, 17),
 (1, 2, 3, 5, 18),
 (1, 2, 3, 5, 19),
 (1, 2, 3, 5, 20),
 (1, 2, 3, 6, 7),
 (1, 2, 3, 6, 8),
 (1, 2, 3, 6, 9),
 (1, 2, 3, 6, 10),
 (1, 2, 3, 6, 11),
 (1, 2, 3, 6, 12),
 (1, 2, 3, 6, 13),
 (1, 2, 3, 6, 14),
 (1, 2, 3, 6, 15),
 (1, 2, 3, 6, 16),
 (1, 2, 3, 6, 17),
 (1, 2, 3, 6, 18),
 (1, 2, 3, 6, 19),
 (1, 2, 3, 6, 20),
 (1, 2, 3, 7, 8),
 (1, 2, 3, 7, 9),
 (1, 2, 3, 7, 10),
 (1, 2, 3, 7, 11),
 (1, 2, 3, 7, 12),
 (1, 2, 3, 7, 13),
 (1, 2, 3, 7, 14),
 (1, 2, 3, 7, 15),
 (1, 2,

In [15]:
np.array(skill_combinations)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (21699,) + inhomogeneous part.

In [6]:
from math import comb

# Total number of skills
n = 50

# Calculating the sum of combinations for r from 1 to 5
total_combinations = sum(comb(n + r - 1, r) for r in range(1, 6))
total_combinations


3478760

In [7]:
n=20
sum(comb(n,k) for k in range(1, 6))

21699